**Νευρωνικά Δίκτυα και Βαθιά Μάθηση - 2η εργαστηριακή άσκηση**

Χριστόδουλος Στυλιανίδης - 03120614

# Installs

In [1]:
! pip install evaluate
! pip install sentence-transformers
! pip install transformers[torch] accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 837.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.w

In [2]:
! pip install datasets

### Προσοχή

Μη διαγράψετε τα # insert your code here σχόλια, καθώς βοηθούν στη διόρθωση. Συμπληρώστε τον κώδικά σας μετά από τα σχόλια αυτά.

# Μέρος Α: Fine-tune a pretrained model

Τα γλωσσικά μοντέλα αποτελούνται από δύο στάδια εκπαίδευσης:
1. Pre-training σε μεγάλα unlabelled datasets. Το pre-training είναι υπολογιστικά πολύ ακριβό και γι αυτό στην πράξη δε το χρησιμοποιούμε όταν θέλουμε να τρέξουμε ένα μοντέλο σε ένα καινούργιο dataset. Μπορούμε να σκεφτούμε το pre-training ως τη διαδικασία εκμάθησης γλωσσικών κανόνων κι εννοιών, οι οποίες στη συνέχεια μπορούν να χρησιμοποιηθούν για διάφορους σκοπούς.

2. Fine-tuning σε μικρότερα labelled datasets. Το fine-tuning πρακτικά εκμεταλλεύεται τις ιδιότητες του transfer learning προκειμένου να μεταφέρουμε τη 'γνώση' που έχει αποθηκευθεί στο γλωσσικό μοντέλο κατά τη διάρκεια του pre-training σε συγκεκριμένα task. Κάθε task εξυπηρετείται μέσω στοχευμένων datasets. Για παράδειγμα, κάποια datasets αναφέρονται στην ταξινόμηση κειμένων σε κατηγιορίες (text classification), άλλα datasets περιέχουν ερωτήσεις οι οποίες πρέπει να απαντηθούν (question answering) κι άλλα πολλά.

Κάποια κλασικά tasks της επεξεργασίας φυσικής γλώσσας είναι τα ακόλουθα:
- Text classification
- Question answering
- Natural language inference
- Fill mask
- Semantic similarity

Περισσότερες πληροφορίες μπορείτε να βρείτε στον ακόλουθο σύνδεσμο στο domain Natural Language Processing: https://huggingface.co/models

Στο πρώτο κομμάτι της παρούσας εργαστηριακής άσκησης, θα χρησιμοποιήσουμε το pre-training fine-tuning σενάριο για να ταξινομήσουμε reviews.

## Pipelines

Με τη χρήση του text-classification pipeline μπορούμε να τρέξουμε γλωσσικά μοντέλα που αφορούν tasks ταξινόμησης. Το natural language inference (NLI) task αποτελεί ένα task ταξινόμησης, αφού το σχετικό μοντέλο (εν προκειμένω το roberta-large-mnli) καλείται να ταξινομήσει ένα κείμενο σε μία από τις 3 κατηγορίες [entailment/neutral/contradiction].

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]
```

Ένα άλλο task ταξινόμησης αφορά την αξιολόγηση του κατά πόσο ένα κείμενο είναι γραμματικά ορθό (acceptable) ή όχι (unacceptable):

```
from transformers import pipeline

classifier = pipeline("text-classification", model = "textattack/distilbert-base-uncased-CoLA")
classifier("I will walk to home when I went through the bus.")
##  [{'label': 'unacceptable', 'score': 0.95}]
```

In [3]:
from transformers import pipeline

classifier = pipeline("text-classification", model = "roberta-large-mnli")
classifier("A soccer game with multiple males playing. Some men are playing a sport.")
## [{'label': 'ENTAILMENT', 'score': 0.98}]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/688 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

[{'label': 'ENTAILMENT', 'score': 0.9883741140365601}]

## Σύνολο δεδομένων Yelp polarity

Κατεβάζουμε το [Yelp Polarity](https://huggingface.co/datasets/yelp_polarity) dataset το οποίο περιέχει reviews που εκφράζουν συναισθήματα πελατών για εστιατόρια. Τα reviews αυτά χωρίζονται σε κατηγορίες, και ο σκοπός μας είναι να κατηγοριοποιήσουμε νέα reviews στις σωστές κατηγορίες.

In [4]:
from datasets import load_dataset

dataset = load_dataset("yelp_polarity")
# insert your code here

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for yelp_polarity contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/yelp_polarity
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Επειδή το σύνολο δεδομένων του Yelp Polarity περιέχει πολλά δείγματα, προκειμένου να επιταχύνουμε τη διαδικασία του fine-tuning συστήνουμε να διατηρήσετε 300 δείγματα από το train set και 300 δείγματα από το test set.

Ελέγξτε τον αριθμό κατηγοριών που υπάρχουν συνολικά στο train και το test set και διατηρήστε ισορροπημένο αριθμό δειγμάτων ανά κατηγορία για τα σύνολα αυτά κατά την επιλογή των 300 δειγμάτων.

In [5]:
# insert your code here

# Function to sample balanced data
def sample_balanced(data, samples_per_category=150):
    # Get unique categories
    categories = set(data['label'])
    balanced_data = []

    # Sample data from each category
    for category in categories:
        # Filter data by category
        category_data = [item for item in data if item['label'] == category]
        # list comprehension etsi omorfa ^^

        balanced_samples = category_data[:samples_per_category]
        balanced_data.extend(balanced_samples)

    return balanced_data

In [6]:
# Applying the sampling function to the train and test sets
train_dataset = sample_balanced(dataset['train'])
test_dataset = sample_balanced(dataset['test'])

In [7]:

print(len(train_dataset),len(test_dataset))


300 300


# Language Models

Η προεπεξεργασία των κειμένων προηγείται της εισόδου τους στα γλωσσικά μοντέλα. Η διαδικασία αυτή επιτελείται μέσω των Tokenizers, τα οποία μετατρέπουν τα tokens εισόδου σε κατάλληλα IDs του λεξιλογίου προεκπαίδευσης, κι έτσι μετατρέπουν το κείμενο σε μορφή που μπορεί να επεξεργαστεί κάποιο μοντέλο Transformer. Η βιβλιοθήκη Huggingface προσφέρει εύκολες και high-level υλοποιήσεις tokenization, τις οποίες συστήνουμε να ακολουθήσετε στη συνέχεια.

Συγκεκριμένα, αρχικοποιούμε τη διαδικασία του tokenization με χρήση του AutoTokenizer. Επιλέγοντας τη μέθοδο from_pretrained λαμβάνουμε έναν tokenizer που αποκρίνεται στην αρχιτεκτονική του μοντέλου που επιθυμούμε να χρησιμοποιήσουμε, παρέχοντας συμβατό tokenization.

Περισσότερες πληροφορίες για το AutoTokenization μπορείτε να βρείτε εδώ:
https://huggingface.co/docs/transformers/model_doc/auto

Αναφορικά με το μοντέλο BERT το οποίο διδαχθήκατε στο εργαστήριο, μπορείτε να δείτε τη διαδικασία [του tokenization και της αρχικοποίησης του μοντέλου](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertTokenizer):

```
from transformers import AutoTokenizer, BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")
```

Στα πλαίσια της άσκησης καλείστε να επιτελέσετε την παραπάνω διαδικασία με *κάποιο άλλο μοντέλο της επιλογής σας από το Huggingface* που να υποστηρίζει τον AutoTokenizer. Το pre-trained μοντέλο που θα επιλέξετε θα πρέπει να διαθέτει υλοποίηση με sequence classification head (κατ αναλογία της μεθόδου BertForSequenceClassification).

Στο επόμενο κελί, φορτώστε το επιλεχθέν μοντέλο με τον αντίστοιχο tokenizer.

(Αγνοήστε πιθανά warnings της μορφής Some weights of the model checkpoint at xxx were not used when initializing...)

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "distilroberta-base"
'''
Stin ousia to distil roberta (DistilRoBERTa) einai kalitero-grigorotero apo to RoBERTa (to opoio einai branch tou BERT)

DistilRoBERTa is a smaller, faster, and lighter version of the RoBERTa (Robustly optimized BERT approach) model,
which itself is a variant of the BERT (Bidirectional Encoder Representations from Transformers) model.
DistilRoBERTa is part of the Distil* family of models created by Hugging Face, which aims to reduce
the size and computational requirements of large transformer-based language models
while retaining most of their performance.
'''

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Now, tokenizer and model are ready to be used for your sequence classification tasks

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Σας παρέχουμε τη συνάρτηση που πραγματοποιεί το tokenization καλώντας τον tokenizer που επιλέξατε. Εφαρμόστε το τόσο στο train, όσο και στο test set.

In [9]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

# Apply the tokenize function to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/560000 [00:00<?, ? examples/s]

Map:   0%|          | 0/38000 [00:00<?, ? examples/s]

Τυπώνοντας το train ή το test set, θα δείτε δύο επιπλέον πεδία 'input_ids' και 'attention_mask'. Βεβαιωθείτε ότι υπάρχουν, άρα και το tokenization έχει επιτευχθεί.

In [10]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 38000
    })
})

In [11]:
#### example

from transformers import AutoTokenizer

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Example text
texts = ["Hello, world!", "Transformers are great for NLP tasks."]

# Tokenizing the text
tokenized_outputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Outputs include 'input_ids' and 'attention_mask'
print(tokenized_outputs['input_ids'])
print(tokenized_outputs['attention_mask'])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tensor([[  101,  7592,  1010,  2088,   999,   102,     0,     0,     0,     0],
        [  101, 19081,  2024,  2307,  2005, 17953,  2361,  8518,  1012,   102]])
tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])


## Χρήση του PyTorch Trainer για fine-tuning

Η κλάση [Trainer](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer) έχει βελτιστοποιηθεί από τους δημιουργούς του Huggingface παρέχοντας πολλές διευκολύνσεις και λιγότερη 'χεράτη' δουλειά. Προτείνουμε να τη χρησιμοποιήσετε ως εναλλακτική του να γράψετε το δικό σας training loop.
Καθώς η Trainer δεν τεστάρει αυτόματα την επίδοση του εκάστοτε μοντέλου κατά την εκπαίδευση, παρέχουμε κατάλληλη συνάρτηση προκειμένου να αποτιμάται το accuracy του μοντέλου σε κάθε εποχή.

In [12]:
import numpy as np
import evaluate
#import torch
from tqdm import tqdm
from transformers import pipeline

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Η κλάση [TrainingArguments](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments) περιέχει όλες τις υπερπαραμέτρους με τις οποίες μπορείτε να πειραματιστείτε κατά τη διαδικασία fine-tuning.


Καλείστε να πειραματιστείτε με διαφορετικές υπερπαραμέτρους όπως το learning rate, batch size κλπ, καθώς επίσης και να ορίσετε optimizer και scheduler για το fine-tuning. Προτείνουμε να εκτελέσετε fine-tuning για μικρό αριθμό εποχών (άλλωστε το μοντέλο είναι ήδη προεκπαιδευμένο).

1. Θα μας δώσετε σε markdown ένα πινακάκι με διαφορετικές υπερπαραμέτρους που δοκιμάσατε και το accuracy που πετύχατε στην τελευταία εποχή.

2. Βάσει των πειραματισμών, πώς επηρεάζουν διαφορετικές υπερπαράμετροι όπως το learning rate και το batch size το fine-tuning του μοντέλου που επιλέξατε? Σχολιάστε και αναλύστε.

In [ ]:
try:
    from accelerate import Accelerator
    print("Accelerate loaded successfully.")
except ImportError as e:
    print("Error loading Accelerate:", e)

from transformers import TrainingArguments

try:
    args = TrainingArguments(
        output_dir="test_trainer",
        evaluation_strategy="epoch",
        per_device_train_batch_size=16,
    )
    print("TrainingArguments initialized successfully.")
except Exception as e:
    print("Failed to initialize TrainingArguments:", str(e))


Accelerate loaded successfully.
TrainingArguments initialized successfully.


In [25]:
from transformers import TrainingArguments, Trainer


# insert your code here
# optimizer

# scheduler

# etc


# vazw mikrotero dataset giati allios trexei gia 18 hours
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(5000))  # Use only 5000 samples for training
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))   # Use only 1000 samples for evaluation

args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="no",        # Turn off evaluation to save time
    save_strategy="no",              # Turn off model saving to save IO time
    logging_dir="logs",

    per_device_train_batch_size=8,   # Further reduced batch size
    learning_rate=5e-1,
    num_train_epochs=3,

    load_best_model_at_end=False,    # No need to load the best model
    report_to="all",
    logging_steps=100,               # Log more frequently to see progress
    dataloader_num_workers=1,        # Reduce number of workers to avoid overhead
    gradient_accumulation_steps=1,   # No accumulation to simplify processing
    disable_tqdm=False               # Keep tqdm to visualize progress
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [26]:
# Create Trainer instance
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics  # Ensure this function is defined as shown previously
)

Then fine-tune your model by calling [train()](https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.Trainer.train):

In [27]:
# Start training
'''
    per_device_train_batch_size=8,   # Further reduced batch size
    learning_rate=5e-1,              # Higher learning rate to speed up training
    num_train_epochs=3,
'''
trained_model=trainer.train()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss
100,60.252000
200,37.465500
300,48.808300
400,42.814200
500,50.024000
600,47.621200
700,42.355600
800,49.446800
900,34.363300
1000,32.393500


In [24]:
# Start training
'''
    per_device_train_batch_size=16,  # Further reduced batch size
    learning_rate=1e-3,
    num_train_epochs=3,
'''
trained_model=trainer.train()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss
100,0.701000
200,0.701600
300,0.698200
400,0.696900
500,0.696600
600,0.695900
700,0.695200
800,0.693400
900,0.693200


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [21]:
# Start training
'''
Conservative Learning with Moderate Batch Size

    per_device_train_batch_size=32,  # Batch size
    learning_rate=5e-4,              # Higher learning rate to speed up training
    num_train_epochs=3,
'''
trained_model=trainer.train()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss
100,0.696500
200,0.695400
300,0.694500
400,0.693800


In [15]:
# Start training
'''
    per_device_train_batch_size=8,  # Further reduced batch size
    learning_rate=1e-3,              # Higher learning rate to speed up training
    num_train_epochs=3,
'''
trained_model=trainer.train()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss
100,0.814400
200,0.732700
300,0.713200
400,0.707800
500,0.709400
600,0.713200
700,0.710400
800,0.697700
900,0.699800
1000,0.701400


In [18]:
# Start training
'''
Conservative Learning with Moderate Batch Size

    per_device_train_batch_size=16,  # Batch size
    learning_rate=5e-4,              # Higher learning rate to speed up training
    num_train_epochs=3,
'''
trained_model=trainer.train()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss
100,0.701100
200,0.698600
300,0.697700
400,0.694900
500,0.694300
600,0.695700
700,0.694400
800,0.693300
900,0.693200


<style>
    /* Increase the font size and width of table */
    table.dataframe, .dataframe th, .dataframe td {
        font-size: 16px; /* Adjust the font size as needed */
        width: 100% !important;
    }
</style>


| Batch Size | Learning Rate | Epochs | Last Step | Training Loss |
|------------|---------------|--------|-----------|---------------|
| 8          | 5e-1          | 3      | 1200      | 12.5516       |
| 8          | 1e-3          | 3      | 1800      | 0.69650       |
| 16         | 1e-3          | 3      | 900       | 0.69320       |
| 16         | 5e-4          | 3      | 900       | 0.69320       |
| 32         | 5e-4          | 3      | 400       | 0.69380       |





*   **Batch Size: 16**
*   **Learning Rate: 1e-3**
*   **Epochs: 3**


Οδηγεί στη χαμηλότερη απώλεια εκπαίδευσης στο τελευταίο βήμα σε σύγκριση με τις άλλες διαμορφώσεις/συνδιασμούς. Αυτό υποδηλώνει ότι ένα μέτριο batch size και ένα χαμηλότερο learning rate, σε συνδυασμό με έναν επαρκή αριθμό epochs, παρέχουν μια καλύτερη ισορροπία μεταξύ σταθερότητας και αποτελεσματικότητας στην εκπαίδευση υπό τις συνθήκες και το μοντέλο που χρησιμοποιείτε. Αυτή η διαμόρφωση φαίνεται να ελαχιστοποιεί αποτελεσματικά την απώλεια εκπαίδευσης, υποδηλώνοντας πιο σταθερή και αποτελεσματική δυναμική μάθησης.

# Μέρος Β: Χρήση fine-tuned μοντέλων σε νέα tasks


Στο κομμάτι αυτό της εργασίας δε χρειάζεται να πραγματοποιήσετε εκπαίδευση σε γλωσσικά μοντέλα. Αντιθέτως, θα εκμεταλλευτούμε τις δυνατότητες του transfer learning για να αντιμετωπίσουμε πιο πολύπλοκα γλωσσικά task, ανάγοντάς τα σε κλασικά task όπως είναι το text classification, natural language inference, question answering και άλλα.

Για παράδειγμα, fine-tuned μοντέλα για [text classification](https://huggingface.co/tasks/text-classification) εξυπηρετούν tasks όπως:

- Είναι δύο προτάσεις η μία παράφραση της άλλης? [Paraphrase/No Paraphrase]
- Συνεπάγεται η πρόταση Χ την πρόταση Υ? [Entail/Neutral/Contradict]
- Είναι η δοθείσα πρόταση γραμματικά ορθή? [Acceptable/Unacceptable]

## B1. Piqa dataset

Το [Piqa dataset](https://huggingface.co/datasets/piqa) περιλαμβάνει προτάσεις οι οποίες ελέγχουν το βαθμό στον οποίο τα language models έχουν κοινή γνώση (commonsense). Συγκεκριμένα, αποτελείται από προτάσεις και πιθανά endings, τα οποία απαιτούν commonsense γνώση για να συμπληρωθούν.

Για παράδειγμα, έχοντας την πρόταση "When boiling butter, when it's ready, you can" υπάρχουν δύο υποψήφια endings:
- "Pour it onto a plate"
- "Pour it into a jar"

Ένας άνθρωπος μπορεί να συμπεράνει ότι η δεύτερη πρόταση αποτελεί ένα πιο κατάλληλο ending, αφού το λιωμένο βούτυρο είναι υγρό, άρα το βάζο είναι ένα καταλληλότερο δοχείο σε σχέση με το πιάτο.

Για λόγους επιτάχυνσης επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το Piqa.

In [28]:
# # insert your code here (load dataset)
from datasets import load_dataset

dataset = load_dataset("piqa")

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for piqa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/piqa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

Μπορούμε να θεωρήσουμε το παραπάνω σενάριο σαν ένα πρόβλημα πολλαπλής επιλογής, όπου υπάρχουν δύο πιθανές εναλλακτικές για το ending της πρότασης. Συνεπώς, αξιοποιώντας σχετικά μοντέλα μπορούμε να επιλύσουμε την επιλογή του ending δοθείσας της πρότασης.

Καλείστε λοιπόν να καταγράψετε το accuracy πρόβλεψης endings για κάθε πρόταση με χρήση γλωσσικών μοντέλων. Για λόγους σύγκρισης χρησιμοποιήστε τουλάχιστον 5 κατάλληλα μοντέλα.

In [29]:
from transformers import AutoModelForMultipleChoice, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import random
import numpy as np

# Function to prepare data in multiple-choice format
# epeksergasia tou piqa dataset gia na mporoume na to kanoume train
def preprocess_function(examples):

    # Unpack the context and endings into separate lists
    first_sentences = [[context] * 2 for context in examples['goal']]
    question_headers = examples['sol1']
    second_sentences = examples['sol2']
    # Flatten out
    first_sentences = sum(first_sentences, [])
    question_headers = sum([[choice1, choice2] for choice1, choice2 in zip(question_headers, second_sentences)], [])

    # Encode
    encoded = tokenizer(first_sentences, question_headers, truncation=True, padding=True, max_length=512)
    # Format for multiple choice
    num_examples = len(examples['goal'])
    features = {k: [v[i:i+2] for i in range(0, len(v), 2)] for k, v in encoded.items()}
    features['labels'] = examples['label']
    return features

# Use a random subset of 100 samples for speeding up the process
small_dataset = dataset['train'].shuffle(seed=42).select(range(100))
models = ['bert-base-uncased', 'roberta-base', 'albert-base-v2', 'distilbert-base-uncased', 'xlnet-base-cased']

results = {}

for model_name in models:
    # Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMultipleChoice.from_pretrained(model_name)
    # Preprocess the data
    processed_dataset = small_dataset.map(preprocess_function, batched=True)
    # Define dummy training arguments - we're only using the Trainer for prediction
    training_args = TrainingArguments(
        output_dir='./results',
        do_train=False,
        do_eval=True,
        per_device_eval_batch_size=8
    )
    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        tokenizer=tokenizer
    )
    # Make prediction
    raw_pred, _, _ = trainer.predict(processed_dataset)
    predictions = np.argmax(raw_pred, axis=1)
    accuracy = (predictions == processed_dataset['labels']).mean()
    results[model_name] = accuracy

# Print the results
for model, acc in results.items():
    print(f"Model: {model}, Accuracy: {acc}")


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer.json:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of XLNetForMultipleChoice were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Model: bert-base-uncased, Accuracy: 0.59
Model: roberta-base, Accuracy: 0.46
Model: albert-base-v2, Accuracy: 0.55
Model: distilbert-base-uncased, Accuracy: 0.47
Model: xlnet-base-cased, Accuracy: 0.47


In [30]:
from transformers import AutoModelForMultipleChoice, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch

# Load tokenizer and model
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMultipleChoice.from_pretrained(model_name)

# Function to predict the best ending
def predict_ending(context, ending1, ending2):
    # Encode the context and the endings into the format expected by the model
    inputs = tokenizer([context, context], [ending1, ending2], return_tensors='pt', padding=True, truncation=True, max_length=512)
    # Multiple choice models expect inputs in a different dimension so we adjust it
    inputs = {k: v.unsqueeze(0) for k, v in inputs.items()}  # Add batch dimension

    # Forward pass, get logits
    outputs = model(**inputs)
    logits = outputs.logits

    # We use softmax to convert logits to probabilities
    probs = torch.softmax(logits, dim=1)

    # Find the index of the highest probability to determine the chosen ending
    best_ending = torch.argmax(probs, dim=1).item()

    # Return the selected ending
    return ending1 if best_ending == 0 else ending2

# Example usage
context = "When boiling butter, when it's ready, you can"
ending1 = "Pour it onto a plate"
ending2 = "Pour it into a jar"

# Predict the best ending
best_ending = predict_ending(context, ending1, ending2)
print(f"The best ending is: '{best_ending}'")


Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The best ending is: 'Pour it into a jar'


## B2. Truthful QA

### Sentence Transformers

Οι sentence transformers χρησιμοποιούνται για να δημιουργήσουν embeddings προτάσεων, δηλαδή διανυσματικές αναπαραστάσεις των προτάσεων αυτών σε ένα διανυσματικό χώρο. Χάρη στον τρόπο που έχουν προεκπαιδευτεί, έχουν την ικανότητα να τοποθετούν νοηματικά όμοιες προτάσεις κοντά τη μία στην άλλη, ενώ απομακρύνουν νοηματικά μακρινές προτάσεις. Έτσι, χάρη στις αναπαραστάσεις που λαμβάνουμε από τα sentence embeddings μπορούμε να αξιολογήσουμε σε τι βαθμό δύο προτάσεις είναι κοντά ή μακριά νοηματικά.

Η σύγκριση των διανυσματικών αναπαραστάσεων μπορεί να γίνει κλασικά μέσω μεθόδων όπως το consine similarity, με μεγαλύτερες τιμές μεταξύ διανυσμάτων να σηματοδοτούν πιο όμοια διανύσματα, άρα και πιο όμοιες προτάσεις. Δίνουμε για το λόγο αυτό μια συνάρτηση υπολογισμού του cosine similarity.

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
def get_cosine_similarity(feature_vec_1, feature_vec_2):
    return cosine_similarity(feature_vec_1.reshape(1, -1), feature_vec_2.reshape(1, -1))[0][0]

Για παράδειγμα, εκτελέστε το ακόλουθο κελί, το οποίο δίνει μια τιμή ομοιότητας στο διάστημα [0, 1] για δύο προτάσεις ("This is an example sentence", "Each sentence is converted"). Μπορείτε ακόμα να δοκιμάσετε να εκτελέσετε το ακόλουθο κελί για διαφορετικές προτάσεις της επιλογής σας, που μπορεί να είναι όμοιες ή πολύ διαφορετικές μεταξύ τους, και να παρατηρήσετε τις αλλαγές τιμών του cosine similarity.

In [32]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings = model.encode(sentences)

get_cosine_similarity(embeddings[0], embeddings[1])

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

0.4048847

Για τη συνέχεια της άσκησης, καλείστε να επιλέξετε τουλάχιστον 6 διαφορετικά [μοντέλα για semantic similarity](https://huggingface.co/models?pipeline_tag=sentence-similarity&sort=downloads) από τους sentence transformers

### Μπορούν τα question answering μοντέλα να διαχωρίσουν αληθείς και ψευδείς προτάσεις?

Αυτό το ερώτημα θα το απαντήσουμε στο παρόν κομμάτι της άσκησης. Για το λόγο αυτό, φορτώνουμε το dataset [Truthful QA generation](https://huggingface.co/datasets/truthful_qa/viewer/generation/validation), το οποίο περιέχει τις εξής επιλογές:

- best answer
- correct answer
- incorrect answer

Πολλές φορές το best answer και το correct answer είναι ίδια ή έστω πολύ κοντινά νοηματικά. Σε αυτό το σημείο είναι που θα αξιοποιήσουμε το semantic similarity για να αξιολογήσουμε την ομοιότητα αυτή.

Φιλτράρουμε το dataset ώστε να περιέχονται 100 δείγματα συνολικά για λόγους επιτάχυνσης, εκ των οποίων καθένα θα πρέπει να περιέχει τουλάχιστον 2 correct answer. Θεωρούμε έτσι 4 υποψήφιες επιλογές:

1η επιλογή: best answer  
2η επιλογή: 1ο correct answer  
3η επιλογή: 2ο correct answer  
4η επιλογή: incorrect answer  

Οι επιλογές αυτές μαζί με την ερώτηση δίνονται σε ένα μοντέλο πολλαπλής επιλογής σαν αυτά που χρησιμοποιήθηκαν στο ερώτημα Β1. Μπορείτε να θεωρήσετε τα ίδια μοντέλα και να τα επεκτείνετε σε 4 υποψήφιες απαντήσεις.  

Το semantic similarity θα επηρεάσει το τι θεωρούμε βέλτιστα σωστή απάντηση, άρα και το accuracy. Συγκεκριμένα, θα λάβουμε διανυσματικές αναπαραστάσεις για το best answer και τα 2 correct answer που έχουν δοθεί ως υποψήφιες επιλογές μέσω κάποιου semantic similarity μοντέλου. Σε περίπτωση λοιπόν που το μοντέλο πολλαπλής επιλογής προβλέψει ένα εκ των correct answer, και η ομοιότητά τους σε σχέση με το best model ξεπερνάει ένα προεπιλεγμένο κατώφλι ομοιότητας, η απάντηση θεωρείται βέλτιστα σωστή. Θέτουμε λοιπόν κατώφλι ομοιότητας το 0.95.

Για παράδειγμα, έστω ότι το μοντέλο πολλαπλής επιλογής μεταξύ των υποψηφίων [best, 1st correct, 2nd correct, incorrect] επιλέγει το δεύτερο στοιχείο, δηλαδή το 1st correct, και δεδομένου ότι το cosine similarity μεταξύ των embeddings του best και του 1st correct είναι > 0.95, τότε θεωρούμε ότι η απάντηση είναι βέλτιστα σωστή, και συνυπολογίζεται θετικά στο accuracy.

Καλείστε λοιπόν να γράψετε μια συνάρτηση που να υπολογίζει το accuracy εύρεσης βέλτιστα σωστών απαντήσεων ανάμεσα στις υποψήφιες απαντήσεις, εξετάζοντας τουλάχιστον 6 semantic similarity μοντέλα καθώς επίσης και τα μοντέλα που επιλέξατε στο ερώτημα Β1.


In [33]:
# insert your code here (load dataset)

from datasets import load_dataset

# Load the Truthful QA dataset
dataset = load_dataset("truthful_qa", "generation", split='validation')

# We need to filter the dataset to include only entries with at least two correct answers and keep only 100 samples for quick processing.
def has_two_correct_answers(example):
    return len(example['correct_answers']) >= 2

filtered_dataset = dataset.filter(has_two_correct_answers).select(range(100))
print("Dataset loaded and filtered.")

Generating validation split:   0%|          | 0/817 [00:00<?, ? examples/s]

Filter:   0%|          | 0/817 [00:00<?, ? examples/s]

Dataset loaded and filtered.


In [34]:
# insert your code here (load models for semantic similarity and QA)

from sentence_transformers import SentenceTransformer
from transformers import AutoModelForMultipleChoice, AutoTokenizer

# Load multiple models for semantic similarity
similarity_model_names = [
    'sentence-transformers/all-MiniLM-L12-v2',
    'sentence-transformers/all-mpnet-base-v2',
    'sentence-transformers/all-distilroberta-v1',
    'sentence-transformers/paraphrase-MiniLM-L6-v2',
    'sentence-transformers/paraphrase-albert-small-v2',
    'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
]

similarity_models = {name: SentenceTransformer(name) for name in similarity_model_names}
print("Semantic similarity models loaded.")

# vazoume ta mondella apo to proigumeno part (epilego to kalitero)
qa_models = {
    'bert-base-uncased': (AutoTokenizer.from_pretrained("bert-base-uncased"), AutoModelForMultipleChoice.from_pretrained("bert-base-uncased")),
}
print("QA models loaded.")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Semantic similarity models loaded.


Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


QA models loaded.


In [35]:
# insert your code here (function for optimal correct answers & semantic similarity)

from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModelForMultipleChoice, pipeline
import torch

def evaluate_accuracy(data, qa_models, similarity_models, threshold=0.85):
    total = 0
    optimally_correct = 0

    for example in data:
        question = example['question']
        choices = [example['best_answer']] + example['correct_answers'][:2] + [example['incorrect_answers'][0]]
        if len(choices) < 4:
            continue  # Ensure there are always 4 choices

        # Process each QA model
        for model_name, (tokenizer, model) in qa_models.items():
            # Pair the question with each choice and tokenize
            input_pairs = [[question, choice] for choice in choices]
            inputs = tokenizer(input_pairs, padding=True, truncation=True, return_tensors="pt", max_length=512)
            # Adjust inputs for multiple choice (reshape to handle the number of choices)
            inputs = {key: value.view(-1, 4, value.size(-1)) for key, value in inputs.items()}

            # Forward pass through the model
            with torch.no_grad():
                outputs = model(**inputs)
                predictions = torch.argmax(outputs.logits, dim=-1)
                selected_answer = choices[predictions.item()]

            # Compute embeddings and cosine similarities for the selected answer
            for sim_model_name, sim_model in similarity_models.items():
                embeddings = sim_model.encode([example['best_answer'], selected_answer])
                similarity = get_cosine_similarity(embeddings[0], embeddings[1])

                if similarity > threshold:
                    optimally_correct += 1
                total += 1

    accuracy = optimally_correct / total if total > 0 else 0
    return accuracy

# Assuming filtered_dataset, qa_models, and similarity_models are correctly set up
accuracy = evaluate_accuracy(filtered_dataset, qa_models, similarity_models)
print(f"Calculated accuracy for optimally correct answers: {accuracy}")
# stin ousia to accuracy pou exoume einai gia tis apantiseis oi opoies einai
# panw apo 85% sigoures

Calculated accuracy for optimally correct answers: 0.465


## Β3. Winogrande dataset

Το [Winogrande dataset](https://huggingface.co/datasets/winogrande) αποτελείται από προτάσεις που μία λέξη τους έχει αφαιρεθεί και δίνονται δύο πιθανές επιλογές συμπλήρωσης του κενού. Για παράδειγμα, δοθείσας της πρότασης "John moved the couch from the garage to the backyard to create space. The _ is small.", υπάρχουν δύο πιθανές εναλλακτικές:

- "garage"
- "backyard"

Η δυσκολία της συμπλήρωσης έγκειται στο ότι και οι δύο λέξεις αναφέρονται στην πρόταση, οπότε το μοντέλο θα πρέπει να διαθέτει υψηλές δυνατότητες κατανόησης γλώσσας προκειμένου να επιλέξει μια νοηματικά σωστή συμπλήρωση.

Για λόγους επιτάχυνσης, επιλέξτε ένα τυχαίο υποσύνολο 100 δειγμάτων από το training set του Winogrande.


In [36]:
# insert your code here (load dataset)

from datasets import load_dataset

# Load the Winogrande dataset from the Huggingface datasets library
dataset = load_dataset("winogrande", "winogrande_xs")

# Randomly shuffle the dataset and select a subset of 100 samples for quick evaluation
filtered_dataset = dataset['train'].shuffle(seed=42).select(range(100))

print("Dataset loaded and filtered. Number of examples:", len(filtered_dataset))


/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for winogrande contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/winogrande
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/160 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1267 [00:00<?, ? examples/s]

Dataset loaded and filtered. Number of examples: 100


Με κατάλληλο [μετασχηματισμό](https://huggingface.co/DeepPavlov/roberta-large-winogrande) της παραπάνω εισόδου (πρόταση με κενό και δύο επιλογές συμπλήρωσης), καλείστε να καταγράψετε το accuracy σχετικών μοντέλων που επιλύουν το πρόβλημα, συγκρίνοντας το predicted label με το πραγματικό label (1: πρώτη επιλογή, 2: δεύτερη επιλογή). Ουσιαστικά θα πρέπει να αναγάγετε το παραπάνω πρόβλημα σε κάποιο πιο κλασικό πρόβλημα της επεξεργασίας φυσικής γλώσσας.

Δοκιμάστε τουλάχιστον 3 κατάλληλα μοντέλα από το Huggingface για να προσεγγίσετε το πρόβλημα του Winogrande. Προτείνουμε τη χρήση pipelines για τη διευκόλυνσή σας.

In [37]:
# insert your code here (load models)

from transformers import AutoModelForMultipleChoice, AutoTokenizer

# Define a list of model names that are suited for the Winogrande dataset task
model_names = [
    'DeepPavlov/roberta-large-winogrande',  # Specifically fine-tuned for the Winogrande task
    'roberta-large',                        # General-purpose large model known for its strong performance on similar tasks
    'google/electra-large-discriminator'    # Another strong model for NLP tasks including multiple choice
]

# Load models and tokenizers
models = {name: (AutoTokenizer.from_pretrained(name), AutoModelForMultipleChoice.from_pretrained(name)) for name in model_names}

print("Models and tokenizers loaded successfully.")



tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of ElectraForMultipleChoice were not initialized from the model checkpoint at google/electra-large-discriminator and are newly initialized: ['classifier.bias', 'classifier.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Models and tokenizers loaded successfully.


In [38]:
# insert your code here (create pipelines)

# While we can't use a pipeline directly for multiple choice in a standard way, we can setup a similar convenience function
def predict_with_model(tokenizer, model, sentence, choices):
    inputs = tokenizer([[sentence.replace('_', choice) for choice in choices]], return_tensors="pt", padding=True)
    outputs = model(**inputs)
    prediction = outputs.logits.argmax(-1).item()
    return choices[prediction]

print("Ready to predict using models.")


Ready to predict using models.


In [39]:
# insert your code here (function for predicting best fill)
import torch
#import torch
#from transformers import AutoTokenizer, AutoModelForMultipleChoice
#from datasets import load_dataset

# Mock-up models dictionary (make sure to load your models correctly)
# models = {
#     'model_name': (AutoTokenizer.from_pretrained("model_checkpoint"), AutoModelForMultipleChoice.from_pretrained("model_checkpoint"))
# }

def evaluate_fill_in_the_blank(data, models):
    correct_predictions = 0
    total_predictions = len(data)

    for example in data:
        sentence = example['sentence']
        options = [example['option1'], example['option2']]
        correct_index = int(example['answer']) - 1  # Adjust for 0-based indexing.

        # Iterate over each model
        for model_name, (tokenizer, model) in models.items():
            # Tokenize each option with the placeholder replaced
            inputs = tokenizer([sentence.replace('_', opt) for opt in options], return_tensors="pt", padding=True, truncation=True, max_length=512)
            # Since we have 2 options, inputs should be adjusted to simulate a batch where each "batch" is a single example
            input_ids = inputs['input_ids'].unsqueeze(0)
            attention_mask = inputs['attention_mask'].unsqueeze(0)

            # Forward pass through the model
            with torch.no_grad():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
                logits = outputs.logits
                predictions = logits.argmax(-1)
                selected_answer_index = predictions.item()

            if selected_answer_index == correct_index:
                correct_predictions += 1

        total_predictions += 1

    accuracy = correct_predictions / total_predictions if total_predictions > 0 else 0
    return accuracy

# Assuming 'filtered_dataset' is a correctly prepared list of dictionaries
accuracy = evaluate_fill_in_the_blank(filtered_dataset, models)
print(f"Calculated accuracy: {accuracy:.2f}")


Calculated accuracy: 0.93
